In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
from pandas.plotting import scatter_matrix

from sklearn import manifold

data_dir = '../input/petfinder-pawpularity-score'

In [ ]:
train = pd.read_csv(data_dir + '/train.csv')
test = pd.read_csv(data_dir + '/test.csv')
sample_submission = pd.read_csv(data_dir + '/sample_submission.csv')

In [ ]:
train.shape, test.shape, sample_submission.shape

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

**Photo Metadata**  
The train.csv and test.csv files contain metadata for photos in the training set and test set, respectively. Each pet photo is labeled with the value of 1 (Yes) or 0 (No) for each of the following features:  

* Focus - Pet stands out against uncluttered background, not too close / far.
* Eyes - Both eyes are facing front or near-front, with at least 1 eye / pupil decently clear.
* Face - Decently clear face, facing front or near-front.
* Near - Single pet taking up significant portion of photo (roughly over 50% of photo width or height).
* Action - Pet in the middle of an action (e.g., jumping).
* Accessory - Accompanying physical or digital accessory / prop (i.e. toy, digital sticker), excluding collar and leash.
* Group - More than 1 pet in the photo.
* Collage - Digitally-retouched photo (i.e. with digital photo frame, combination of multiple photos).
* Human - Human in the photo.
* Occlusion - Specific undesirable objects blocking part of the pet (i.e. human, cage or fence). Note that not all blocking objects are considered occlusion.
* Info - Custom-added text or labels (i.e. pet name, description).
* Blur - Noticeably out of focus or noisy, especially for the pet’s eyes and face. For Blur entries, “Eyes” column is always set to 0.

In [ ]:
train.isnull().sum()

below_original code (very thanks subinium) : https://www.kaggle.com/subinium/tps-may-categorical-eda

In [ ]:
from cycler import cycler

raw_light_palette = [
    (0, 122, 255), # Blue
    (255, 149, 0), # Orange
    (52, 199, 89), # Green
    (255, 59, 48), # Red
    (175, 82, 222),# Purple
    (255, 45, 85), # Pink
    (88, 86, 214), # Indigo
    (90, 200, 250),# Teal
    (255, 204, 0)  # Yellow
]

raw_dark_palette = [
    (10, 132, 255), # Blue
    (255, 159, 10), # Orange
    (48, 209, 88),  # Green
    (255, 69, 58),  # Red
    (191, 90, 242), # Purple
    (94, 92, 230),  # Indigo
    (255, 55, 95),  # Pink
    (100, 210, 255),# Teal
    (255, 214, 10)  # Yellow
]

raw_gray_light_palette = [
    (142, 142, 147),# Gray
    (174, 174, 178),# Gray (2)
    (199, 199, 204),# Gray (3)
    (209, 209, 214),# Gray (4)
    (229, 229, 234),# Gray (5)
    (242, 242, 247),# Gray (6)
]

raw_gray_dark_palette = [
    (142, 142, 147),# Gray
    (99, 99, 102),  # Gray (2)
    (72, 72, 74),   # Gray (3)
    (58, 58, 60),   # Gray (4)
    (44, 44, 46),   # Gray (5)
    (28, 28, 39),   # Gray (6)
]

light_palette = np.array(raw_light_palette)/255
dark_palette = np.array(raw_dark_palette)/255
gray_light_palette = np.array(raw_gray_light_palette)/255
gray_dark_palette = np.array(raw_gray_dark_palette)/255

mpl.rcParams['axes.prop_cycle'] = cycler('color',dark_palette)
mpl.rcParams['figure.facecolor']  = gray_dark_palette[-2]
mpl.rcParams['figure.edgecolor']  = gray_dark_palette[-2]
mpl.rcParams['axes.facecolor'] =  gray_dark_palette[-2]

white_color = gray_light_palette[-2]
mpl.rcParams['text.color'] = white_color
mpl.rcParams['axes.labelcolor'] = white_color
mpl.rcParams['axes.edgecolor'] = white_color
mpl.rcParams['xtick.color'] = white_color
mpl.rcParams['ytick.color'] = white_color

mpl.rcParams['figure.dpi'] = 200

mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

In [ ]:
# Heatmap for the entire data

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
train_corr = train.corr()

mask = np.zeros_like(train_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(train_corr, ax=ax,
           square=True, center=0, linewidth=1,
           cmap=sns.diverging_palette(240, 10, as_cmap=True),
           cbar_kws={'shrink': .82},
           mask=mask,
           annot=True,
           annot_kws={'size':7}
           )
ax.set_title(f'Correlation', loc='left', fontweight='bold')
plt.show()

In [ ]:
df = train.drop(['Pawpularity'], axis=1)
df_label = train['Pawpularity'].copy()

In [ ]:
# correlation > 0.099

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)
plt.figure(figsize=(14, 6))
plt.rc('font', size=10)

sns.scatterplot(data=df, x='Subject Focus', y=df_label, alpha=0.1, ax=ax0)
sns.scatterplot(data=df, x='Accessory', y=df_label, alpha=0.1, ax=ax1)
sns.scatterplot(data=df, x='Group', y=df_label, alpha=0.1, ax=ax2)
sns.scatterplot(data=df, x='Blur', y=df_label, alpha=0.1, ax=ax3)

fig.tight_layout()
plt.show()

In [ ]:
# correlation < 0.099 : group 1 

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)
plt.figure(figsize=(14, 6))
plt.rc('font', size=10)

sns.scatterplot(data=df, x='Eyes', y=df_label, alpha=0.1, ax=ax0)
sns.scatterplot(data=df, x='Face', y=df_label, alpha=0.1, ax=ax1)
sns.scatterplot(data=df, x='Near', y=df_label, alpha=0.1, ax=ax2)
sns.scatterplot(data=df, x='Action', y=df_label, alpha=0.1, ax=ax3)

fig.tight_layout()
plt.show()

In [ ]:
# correlation < 0.099 : group 2

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)
plt.figure(figsize=(14, 6))
plt.rc('font', size=10)

sns.scatterplot(data=df, x='Collage', y=df_label, alpha=0.1, ax=ax0)
sns.scatterplot(data=df, x='Human', y=df_label, alpha=0.1, ax=ax1)
sns.scatterplot(data=df, x='Occlusion', y=df_label, alpha=0.1, ax=ax2)
sns.scatterplot(data=df, x='Info', y=df_label, alpha=0.1, ax=ax3)

fig.tight_layout()
plt.show()

In [ ]:
sns.kdeplot(x=train['Pawpularity'])

In [ ]:
train_100 = train[train['Pawpularity'] > 90]
train_100.head()

In [ ]:
attributes = ['Subject Focus', 'Blur', 'Action', 'Near', 'Collage', 'Info']
attributes_corr = ['Subject Focus', 'Accessory', 'Group', 'Blur']

scatter_matrix(train_100[attributes_corr], figsize=(12, 8))

In [ ]:
train_100_label = train_100['Pawpularity']
train_100 = train_100[attributes_corr]

In [ ]:
# Dimension reduction Visualization 

tsne = manifold.TSNE(n_components=2, random_state=42)
transformed_data = tsne.fit_transform(train_100)

tsne_df = pd.DataFrame(np.column_stack((transformed_data, train_100_label)),
                      columns=['X', 'y', 'label']
                      )
tsne_df.loc[:, 'label'] = tsne_df['label'].astype(int)

In [ ]:
tsne_df.head()

### F..Fireworks?

In [ ]:
# Cluster Visualization 

grid = sns.FacetGrid(tsne_df, hue='label', size=8)
grid.map(plt.scatter, "X", 'y').add_legend()

In [ ]:
train_30 = train[(train['Pawpularity'] >= 20) & (train['Pawpularity'] <= 40)]
train_30_label = train_30['Pawpularity']
train_30 = train_30[attributes_corr]

In [ ]:
# Dimension reduction Visualization 

tsne = manifold.TSNE(n_components=2, random_state=42)
transformed_data = tsne.fit_transform(train_30)

tsne_df = pd.DataFrame(np.column_stack((transformed_data, train_30_label)),
                      columns=['X', 'y', 'label']
                      )
tsne_df.loc[:, 'label'] = tsne_df['label'].astype(int)

In [ ]:
# Cluster Visualization (40 > Pawpularity > 30)

grid = sns.FacetGrid(tsne_df, hue='label', size=8)
grid.map(plt.scatter, "X", 'y').add_legend()

Several datas with different characteristics are shown in the section where the Pawpularity score is most distributed. (30, 100)

Original Code (Grandmaster, Tensor girl) : https://www.kaggle.com/usharengaraju/tensorflow-probability-ngboost-w-b

In [ ]:
features = [
    f for f in train.columns if f not in ('Id', 'Pawpularity')
]
train_features = train[features]
df_train = train_features.melt(value_vars=features)

plt.figure(figsize=(15,7))
sns.countplot(data=df_train, y='variable', hue='value')
plt.show()